# To do list
- switch to storage
- ingest csvs like storage class
- figure out traversals

In [1]:
import graph as g
import pandas as pd
from dotenv import load_dotenv, find_dotenv
import os
import storage as s

/Users/noelthomas/Documents/GitHub/Bridge/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv(find_dotenv())

True

In [3]:
data_path = "../../data/datasets/football/"

In [4]:
csv_files = os.listdir(data_path)

In [5]:
storage = s.Storage()

In [6]:
storage.graph.delete_all()

In [7]:
result_list = [data_path + item for item in csv_files]

In [8]:
result_list

['../../data/datasets/football/teams.csv',
 '../../data/datasets/football/appearances.csv',
 '../../data/datasets/football/shots.desc',
 '../../data/datasets/football/appearances.desc',
 '../../data/datasets/football/leagues.desc',
 '../../data/datasets/football/players.desc',
 '../../data/datasets/football/games.desc',
 '../../data/datasets/football/teamstats.csv',
 '../../data/datasets/football/leagues.csv',
 '../../data/datasets/football/players.csv',
 '../../data/datasets/football/games.csv',
 '../../data/datasets/football/teams.desc',
 '../../data/datasets/football/shots.csv',
 '../../data/datasets/football/teamstats.desc']

In [9]:
csv_files = [f for f in result_list if f.endswith('.csv')]

In [10]:
storage.store(csv_files[0])

In [11]:
storage

Graph: 
Vector Storage: 
.... storing 1 value(s)

In [ ]:
csv_files

In [12]:
storage.store_list(csv_files)

In [13]:
storage.create_relationships_structured(result_list)

Skipping ../../data/datasets/football/shots.desc as it's a description file
Skipping ../../data/datasets/football/appearances.desc as it's a description file
Skipping ../../data/datasets/football/leagues.desc as it's a description file
Skipping ../../data/datasets/football/players.desc as it's a description file
Skipping ../../data/datasets/football/games.desc as it's a description file
Skipping ../../data/datasets/football/teams.desc as it's a description file
Skipping ../../data/datasets/football/teamstats.desc as it's a description file


In [17]:
rt = storage.vec_store.query("How many appearances did each player have?")

In [19]:
for index, dictionary in enumerate(rt):
    # Use dict.get() to safely get the value of the 'name' key if it exists,
    # or None if it doesn't exist
    name_value = dictionary.get('name')
    
    if name_value is not None:
        print(f'Dictionary {index}: Name is {name_value}')
    else:
        print(f'Dictionary {index} does not have a "name" key.')

Dictionary 0: Name is appearances
Dictionary 1: Name is players
Dictionary 2: Name is teamstats


In [18]:
storage.graph.node_traversal("games" , "players", 2)

{'path0': ['games.gameID = appearances.gameID',
  'appearances.playerID = players.playerID'],
 'path1': ['games.leagueID = appearances.leagueID',
  'appearances.playerID = players.playerID']}

In [ ]:
print(storage.graph.node_traversal("games" , "players", 3)[0][0][0].end_node)
storage.graph.node_traversal("games" , "players", 3)[0][0][0].start_node

In [ ]:
results, summary, path = storage.graph.node_traversal("games" , "players", 3)

In [ ]:
results[0]

In [ ]:
Path = results[0].value(0)

In [ ]:
relation1, relation2, relation3 = Path.relationships

In [ ]:
relation1.get("key")

In [ ]:
relation3.get("key")

In [ ]:
relation2.get("key")

In [ ]:
for relationship in relationships:
    relationship_properties = relationship.properties
    print("Relationship properties:", relationship_properties)

In [ ]:
relationships = path.relationships

# Iterate through the relationships
for relationship in relationships:
    # Access relationship properties
    relationship_properties = relationship.properties
    print("Relationship properties:", relationship_properties)

In [ ]:
storage.graph.node_traversal("games" , "players", 3)[0][0][0].relationships[0].nodes

In [ ]:
storage.graph.node_traversal("games" , "players", 3)[0][0][0].relationships[0].nodes

In [ ]:
graph = g.Graph(os.getenv('GRAPH_URI'), os.getenv('GRAPH_USER') , os.getenv('GRAPH_PASS'))


In [ ]:
csv_files

In [ ]:
x = s.metadata("../../dev/data/Football Example/" + csv_files[0])

In [ ]:
x.name

In [ ]:
x

In [ ]:
# m = metadata("../../dev/data/Football Example/" + csv_files[0])
# "../../dev/data/Football Example/apperances.csv"

# Pull all test CSVs
- pull all test dfs into a dict/hash map, pull their columns
- pull their PK/FK pairs
  - all PKs are first column
  - check to see if FK appears in column list
  - write another check to find all PK/FKs in dataframes based on keywords ('ID', 'KEY')
    - check across all dataframes for matching keys

- example
  - dataframe w/ GameID
  - dataframe w/ PlayerID
  - dataframe w/ GameID, PlayerID, etc
    - this would be composite key if multiple IDs
  - dataframe w/ GameID (FK), PlayerID (FK), EventID (PK)
    - has one PK, how would we identify?
    - would it be same as node name + id/key?
- non-automated example
  - make a file that makes PK/FK pairs w/ table names
    - relationships.txt
    - relationships.json
- 

In [ ]:
df_dict={}
for f in csv_files:
      
    # read the csv file
    df = pd.read_csv("../../dev/data/Football Example/" + f, encoding='latin-1')
    df_name = f.split("\\")[-1].split('.csv')[0]
    df_keys = []
    for column in df.columns.tolist():
        if ("id") in column.lower():
            df_keys.append(column)
            
    df_dict[df_name] = {"columns" : df.columns.tolist(),
                        "keys": df_keys}
                        
      
    # print the location and filename
    print('Location:', f)
    print('File Name:', f.split("\\")[-1].split('.csv')[0])
      
    # print the content
    # print('Content:')
    # display(df)
    # print()

In [ ]:
print(df_dict.get("appearances"))

# Turn Data Frames into Nodes
- assuming after turning all file types/databases into pandas dataframes we can turn them into nodes
  - also assuming we might not even have to pull full DFs, just metadata, columns, etc.

In [ ]:
graph.delete_all()

In [ ]:
df_dict.items()

In [ ]:
for name, dict in df_dict.items():
    print(name, dict.get("keys"))
    keys = dict.get("keys")
    print(name, dict.get("columns"))
    columns = dict.get("columns")
    print("Creating node!!!")
    graph.add_table_node(name, dict.get("keys"), dict.get("columns"), "TABLE")

In [ ]:
table_relationships = []
for table1, dict1 in df_dict.items():
    keys1 = dict1.get("keys")
    for table2, dict2 in df_dict.items():
        keys2 = dict2.get("keys")
        if table1 != table2:
            shared_keys = list(set(keys1) & set(keys2))
            if shared_keys:
                for i, shared_key in enumerate(shared_keys, start = 1):
                    table_relationships.append((table1, table2, f"{table1}.{shared_key} = {table2}.{shared_key}", i))

In [ ]:
# These relationships weren't recognized, as shooterID and assisterID doesn't explicely say person, bet contextually we know shooters and assisters are people
table1, table2 = 'shots', 'players'
table_relationships.append((table1, table2, f"{table1}.shooterID = {table2}.playerID", i)) 
table_relationships.append((table1, table2, f"{table1}.assisterID = {table2}.playerID", i))

In [ ]:
table_relationships


In [ ]:
for relationship in table_relationships:
    node_one = relationship[0]
    node_two = relationship[1] 
    key = f"""JOIN {{key : "{relationship[2]}"}}"""
    # print(node_one, node_two, key)
    graph.add_relationship(name_node_one= node_one, name_node_two= node_two, relation_name= key)